<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Bayesian_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-optimize

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from skopt import gp_minimize
from skopt.space import Real, Integer

# Define a simple neural network with a variable number of layers
class DynamicNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers):
        super(DynamicNN, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_size, hidden_size))
        for _ in range(n_layers - 1):
            self.layers.append(nn.Linear(hidden_size, hidden_size))
        self.layers.append(nn.Linear(hidden_size, output_size))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = torch.relu(layer(x))
        x = self.layers[-1](x)
        return x

# Dummy data
input_size = 10
hidden_size = 50
output_size = 1
inputs = torch.randn(100, input_size)
targets = torch.randn(100, output_size)

# Define the objective function for optimization
def objective(params):
    lr, n_layers = params
    n_layers = int(n_layers)  # Convert layers to integer

    # Create the model
    model = DynamicNN(input_size, hidden_size, output_size, n_layers)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(50):  # Reduced epochs for faster optimization
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    return loss.item()

# Define the search space for hyperparameters
space = [Real(1e-6, 1e-2, prior='log-uniform', name='lr'),
         Integer(1, 5, name='n_layers')]

# Perform Bayesian optimization
res = gp_minimize(objective, space, n_calls=50, random_state=0)

print(f"Best parameters: {res.x}, Best loss: {res.fun}")